In [ ]:
import os

In [ ]:
os.environ

In [ ]:
migs= 'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'

In [ ]:
env = dict(os.environ)

In [ ]:
env['CUDA_VISIBLE_DEVICES']=migs

In [ ]:
os.environ = env

In [ ]:
os.environ

In [1]:
import os
import ray

# Set CUDA_VISIBLE_DEVICES to control which GPUs are visible to Ray
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6"#'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'#"0,1,2,3,4,5,6"

# Initialize Ray with the appropriate configuration
ray.init()


2023-02-28 15:09:39,299	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.9.16
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8266


In [2]:
resources = ray.cluster_resources()
print(resources)


{'memory': 5913897371.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'object_store_memory': 2956948684.0, 'CPU': 20.0}


In [3]:
import numpy as np
import gymnasium as gym
from gym_CTgraph import CTgraph_env
from gym_CTgraph.CTgraph_plot import CTgraph_plot
from gym_CTgraph.CTgraph_conf import CTgraph_conf
from gym_CTgraph.CTgraph_images import CTgraph_images
import json
import argparse
#import json
import random
import matplotlib.pyplot as plt
import timeit

from gym.spaces import Box, Discrete, Dict
from ray.tune.registry import register_env
from ray import tune

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import argparse
#import gym
from gym.spaces import Discrete, Box
import numpy as np
#import os
import random

import ray
from ray import tune
from ray.rllib.agents import ppo
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.logger import pretty_print

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [4]:
'''import ray
ray.init()
resources = ray.cluster_resources()
print(resources)'''


'import ray\nray.init()\nresources = ray.cluster_resources()\nprint(resources)'

In [5]:
from ray import tune

In [6]:
def env_creator(env_config={}):
    # fetch the parameters from the json file
    configuration = CTgraph_conf("C:\CT-Graph_Native\CT-graph\CT_Graph_Christos\graph09.json")
    conf_data = configuration.getParameters()
    # print configration data
    print(json.dumps(conf_data, indent=3))
    #env = gym.make('CTgraph-v0')
    # instantiate the maze
    #start = timeit.timeit()
    #env = gym.make('CTgraph-v0')
    #end = timeit.timeit()
    #print(end - start)

    imageDataset = CTgraph_images(conf_data)
    env = gym.make('CTgraph-v0', conf_data=conf_data, images=imageDataset)
    env.reset()

    # get a random path from the maze
    #high_reward_path = np.array(np.random.randint(1, conf_data['graph_shape']['branching_factor'], size=conf_data['graph_shape']['depth']))
    # use this random path to set the path to the high reward. Note that the maze would have already a high_reward_path from the initialisation
    #env.set_high_reward_path(high_reward_path)
    #seed_everything(env)  
    
    return env

In [7]:
register_env("CT-Graph_d2_b2_wp05_crv0_MDP", env_creator)

In [8]:
tune.run("DDPPO",
             # algorithm specific configuration
             config={"env": "CT-Graph_d2_b2_wp05_crv0_MDP",  #
                     "framework": "torch",
                     "seed":959,
                     #"num_gpus": 7,
                     "num_rollout_workers": 8,
                     "num_envs_per_worker": 1,
                     "num_gpus_per_worker": 0,
                     "train_batch_size": 512,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 16,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.00015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2
                     },
             local_dir="CT-Graph_d2_b2_wp05_crv0_MDP06",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"timesteps_total": 1500000}
             )


2023-02-28 15:09:55,182	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=38152) WARNING:tensorflow:From C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\tensorflow\python\util\deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=38152) Instructions for updating:
(pid=38152) experimental_relax_shapes is deprecated, use reduce_retracing instead
(DDPPO pid=38152) 2023-02-28 15:10:06,802	WARNING algorithm_config.py:596 -- Cannot create DDPPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(DDPPO pid=38152) 2023-02-28 15:10:06,802	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num

(RolloutWorker pid=7048) ---------------------------------------------------
(RolloutWorker pid=7048)              The CT-graph environments             
(RolloutWorker pid=7048) ---------------------------------------------------
(RolloutWorker pid=7048) Reading configuration parameters from  C:\CT-Graph_Native\CT-graph\CT_Graph_Christos\graph09.json
(RolloutWorker pid=7048) {
(RolloutWorker pid=7048)    "general_seed": 3,
(RolloutWorker pid=7048)    "graph_shape": {
(RolloutWorker pid=7048)       "depth": 2,
(RolloutWorker pid=7048)       "branching_factor": 2,
(RolloutWorker pid=7048)       "wait_prob": 0.5
(RolloutWorker pid=7048)    },
(RolloutWorker pid=7048)    "reward": {
(RolloutWorker pid=7048)       "high_reward_value": 1.0,
(RolloutWorker pid=7048)       "crash_reward_value": 0.0,
(RolloutWorker pid=7048)       "stochastic_sampling": false,
(RolloutWorker pid=7048)       "reward_std": 0.1,
(RolloutWorker pid=7048)       "min_static_reward_episodes": 5000,
(RolloutWorker pid

(RolloutWorker pid=52320) C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\gymnasium\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (12, 12)
(RolloutWorker pid=52320)   logger.warn(
(RolloutWorker pid=24472) C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\gymnasium\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (12, 12)
(RolloutWorker pid=24472)   logger.warn(


(RolloutWorker pid=52320) ---------------------------------------------------
(RolloutWorker pid=52320)              The CT-graph environments             
(RolloutWorker pid=52320) ---------------------------------------------------
(RolloutWorker pid=52320) Reading configuration parameters from  C:\CT-Graph_Native\CT-graph\CT_Graph_Christos\graph09.json
(RolloutWorker pid=52320) {
(RolloutWorker pid=52320)    "general_seed": 3,
(RolloutWorker pid=52320)    "graph_shape": {
(RolloutWorker pid=52320)       "depth": 2,
(RolloutWorker pid=52320)       "branching_factor": 2,
(RolloutWorker pid=52320)       "wait_prob": 0.5
(RolloutWorker pid=52320)    },
(RolloutWorker pid=52320)    "reward": {
(RolloutWorker pid=52320)       "high_reward_value": 1.0,
(RolloutWorker pid=52320)       "crash_reward_value": 0.0,
(RolloutWorker pid=52320)       "stochastic_sampling": false,
(RolloutWorker pid=52320)       "reward_std": 0.1,
(RolloutWorker pid=52320)       "min_static_reward_episodes": 5000,
(

(RolloutWorker pid=40688) C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\gymnasium\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (12, 12)
(RolloutWorker pid=40688)   logger.warn(


...

(pid=35052) WARNING:tensorflow:From C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\tensorflow\python\util\deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=35052) Instructions for updating:
(pid=35052) experimental_relax_shapes is deprecated, use reduce_retracing instead
(pid=50116) WARNING:tensorflow:From C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\tensorflow\python\util\deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=50116) Instructions for updating:
(pid=50116) experimental_relax_shapes is deprecated, use reduce_retracing instead
(pid=16460) WARNING:tensorflow:From C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\tensorflow\python\util\deprecation.py:561: calling function (from tensorflow.python.eager.def_funct

(RolloutWorker pid=35052) ---------------------------------------------------
(RolloutWorker pid=35052)              The CT-graph environments             
(RolloutWorker pid=35052) ---------------------------------------------------
(RolloutWorker pid=35052) Reading configuration parameters from  C:\CT-Graph_Native\CT-graph\CT_Graph_Christos\graph09.json
(RolloutWorker pid=35052) {
(RolloutWorker pid=35052)    "general_seed": 3,
(RolloutWorker pid=35052)    "graph_shape": {
(RolloutWorker pid=35052)       "depth": 2,
(RolloutWorker pid=35052)       "branching_factor": 2,
(RolloutWorker pid=35052)       "wait_prob": 0.5
(RolloutWorker pid=35052)    },
(RolloutWorker pid=35052)    "reward": {
(RolloutWorker pid=35052)       "high_reward_value": 1.0,
(RolloutWorker pid=35052)       "crash_reward_value": 0.0,
(RolloutWorker pid=35052)       "stochastic_sampling": false,
(RolloutWorker pid=35052)       "reward_std": 0.1,
(RolloutWorker pid=35052)       "min_static_reward_episodes": 5000,
(

(DDPPO pid=38152) C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\gymnasium\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (12, 12)
(DDPPO pid=38152)   logger.warn(
(DDPPO pid=38152) 2023-02-28 15:10:33,850	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(DDPPO pid=38152) 2023-02-28 15:10:33,941	INFO trainable.py:172 -- Trainable.setup took 26.534 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(DDPPO pid=38152) 2023-02-28 15:10:33,942	WARNING util.py:67 -- Install gputil for GPU syst

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,evaluation,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
DDPPO_CT-Graph_d2_b2_wp05_crv0_MDP_f5385_00000,1503232,"{'ObsPreprocessorConnector_ms': 0.0072447566298751145, 'StateBufferConnector_ms': 0.007541813291944899, 'ViewRequirementAgentConnector_ms': 0.10424820152488914}","{'num_env_steps_sampled': 1503232, 'num_env_steps_trained': 1503232, 'num_agent_steps_sampled': 1503232, 'num_agent_steps_trained': 1503232}",{},2023-02-28_16-29-39,True,9.27027,{},1,0.268018,0,444,163868,"{'episode_reward_max': 1.0, 'episode_reward_min': 1.0, 'episode_reward_mean': 1.0, 'episode_len_mean': 8.5, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1.0, 1.0], 'episode_lengths': [10, 7]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.4498668733501177, 'mean_inference_ms': 1.239651628658694, 'mean_action_processing_ms': 0.13033091861100177, 'mean_env_wait_ms': 0.4079348322106193, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.0, 'StateBufferConnector_ms': 0.0, 'ViewRequirementAgentConnector_ms': 0.15001296997070312}, 'num_agent_steps_sampled_this_iter': 17, 'num_env_steps_sampled_this_iter': 17, 'timesteps_this_iter': 17, 'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}",d8108ad7b2c5421ca6cb5216e50308e8,Trinity,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.026662068557925522, 'grad_gnorm': 4.133209025894757, 'cur_kl_coeff': 0.0, 'cur_lr': 0.00015, 'total_loss': 0.13693650890604658, 'policy_loss': -0.0020174667931769363, 'vf_loss': 0.13898843613924328, 'vf_explained_var': -0.4583394752116874, 'kl': 0.0, 'entropy': 0.22974297634573304, 'entropy_coeff': 0.00015}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 16.0, 'num_grad_updates_lifetime': 93824.5, 'diff_num_grad_updates_vs_sampler_policy': 127.5}}, 'num_env_steps_sampled': 1503232, 'num_env_steps_trained': 1503232, 'num_agent_steps_sampled': 1503232, 'num_agent_steps_trained': 1503232}",368,127.0.0.1,1503232,1503232,1503232,4096,1503232,4096,0,8,8,0,4096,"{'cpu_util_percent': 5.921428571428572, 'ram_util_percent': 70.81428571428569}",38152,{},{},{},"{'mean_raw_obs_processing_ms': 0.7085340670080695, 'mean_inference_ms': 1.7879923288296968, 'mean_action_processing_ms': 0.1884858015572915, 'mean_env_wait_ms': 0.5848222629252824, 'mean_env_render_ms': 0.0}",4704.41,9.61371,4704.41,"{'training_iteration_time_ms': 9608.649, 'learn_time_ms': 8541.925, 'sample_time_ms': 1030.882}",1677601779,0,1503232,368,f5385_00000,26.5569


2023-02-28 15:36:10,140	WARNING trial_runner.py:964 -- Trial Runner checkpointing failed: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\CT-Graph_Native\\CT-graph\\CT_Graph_Christos\\CT-Graph_d2_b2_wp05_crv0_MDP06\\DDPPO\\.tmp_checkpoint' -> 'C:\\CT-Graph_Native\\CT-graph\\CT_Graph_Christos\\CT-Graph_d2_b2_wp05_crv0_MDP06\\DDPPO\\experiment_state-2023-02-28_15-09-55.json'
2023-02-28 15:59:56,692	WARNING trial_runner.py:964 -- Trial Runner checkpointing failed: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\CT-Graph_Native\\CT-graph\\CT_Graph_Christos\\CT-Graph_d2_b2_wp05_crv0_MDP06\\DDPPO\\.tmp_checkpoint' -> 'C:\\CT-Graph_Native\\CT-graph\\CT_Graph_Christos\\CT-Graph_d2_b2_wp05_crv0_MDP06\\DDPPO\\experiment_state-2023-02-28_15-09-55.json'
2023-02-28 16:00:15,290	WARNING trial_runner.py:964 -- Trial Runner checkpointing failed: [WinError 32] The process cannot access the file because